In [ ]:
# Get Information on where your file system is located (there may be a way to get the full path, as it is /content/mydata/MyDrive)
import os
os.getcwd()

'/content'

In [ ]:

# Import virtual disk drive
from google.colab import drive
 # Mount Drive
drive.mount('/content/mydata')

Mounted at /content/mydata


In [ ]:
# Set path to your to your exiting file folder "MTRL 361" located on google drive (this allows you to avoid typing the full path each time)
path = '/content/mydata/MyDrive/MTRL_361/'

In [ ]:
from locale import T_FMT_AMPM
# Layer 1 - Assignment 4 1D FDM Transient Heat Condution Explicit Formulation
# Dirichlet Condition

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

n_nodes = 40

#Load nodal coordinate array and element connnectivities array

ingot_thickness = 0.200
x_node = np.linspace(0,0.200,n_nodes)
dx=ingot_thickness/(n_nodes)

#print(dx)
#print (x_node)

#Define Material Properties (temperature independent)

Cp_steelNominal = 470 #J/kg/K
k_steel = 35 #W/m/K
rho_steel = 7840 #kg/m^2
h_steel = 2000 #W/m^2/K
dH_Ls = 250000 #J/Kg

#Set Initial Condition and Dirchlet Boundary Condition

T_cast_init = 1500 #C
T_env = 153 #C

T_liquidus = 1522 #C
T_solidus = 1485 #C

T_init_superheat = T_liquidus + 30 #C

#Set time step and total time

t_step = 0.25#s
t_final = 300#s

t_step_check = 1/2*dx**2*rho_steel*Cp_steelNominal/k_steel
print('t_step check =', t_step_check,'s', 'Actual Timestep is =', t_step)

#Calculate the Biot's number and Cpeff for use between T_solidus-T_liquidus

B_i = h_steel*dx/k_steel #doesn't change

Cp_eff = Cp_steelNominal + dH_Ls/(T_liquidus - T_solidus)

#Initialize current time and future time Temperature Vectors to 0 to create the arrays

T_t=np.zeros([n_nodes])
T_tdt=np.zeros([n_nodes])

#Make matrices

A = np.zeros([n_nodes, n_nodes])
B = np.zeros([n_nodes])

# Initialize the intial temperature array to the initial temperature T_init_superheat

for i in range(n_nodes):
  T_t[i] = T_init_superheat

#End Initialization loop

t_total = 0

# Enter time Integration Loop
#k_n = 0
while t_total<=t_final:


# Enter do loop for processing all nodes from LHS of domain x = 0 t0 RHS at x = 0.2m
  for i in range(n_nodes):

#Fourier is no longer constant

    if T_t[i] >= T_liquidus or T_t[i] <= T_solidus:
      #Cp_steel = Cp_steelNominal
      F_0n = k_steel*t_step/(dx**2)/rho_steel/Cp_steelNominal #move F0 into loop, dep on temp based on cp. If T is outside of range liquidus to solidus, use nominal cp

    elif T_solidus< T_t[i] and T_t[i] < T_liquidus:
      #Cp_steel = Cp_eff
      F_0n = k_steel*t_step/(dx**2)/rho_steel/Cp_eff #move into loop, dep on temp based on cp

    if i<(n_nodes-1):
#Vector
      B[i] = T_t[i]

    if i==(n_nodes-1):
      B[i] = T_t[i]+2*F_0n*B_i*T_env

    for j in range(n_nodes):

#First row
      if i==0 and j==0:
        A[i,j] = (2*F_0n+1)
      if i==0 and j==1:
        A[i,j] = -2*F_0n

# All interior nodes
      elif i>0 and i<(n_nodes-1) and j==i:
        A[i,j-1] = -F_0n
        A[i,j] = (1+2*F_0n)
        A[i,j+1] = -F_0n

#External boundary node
    if i==n_nodes-1 and j==(n_nodes-1):
      A[i,j-1] = -2*F_0n
      A[i,j] = (1+2*F_0n*(1+B_i))

#End loop for processing all node

#Solve System of Equations for temperature at future timestep (T_tdt) in Python
  T_tdt = np.linalg.solve(A,B)

#Print out quantities at current t_step
  T_t = T_tdt

#update total time and temperature
  t_total += t_step

# Output array containing T and x data for graphing in Excel

# if divisible by 30 (with no remainder) then output frame this yields and output every 30S
# change to smaller number to output more frequently
  if t_total%41.5 == 0:
    np.savetxt(path+'time_' + str(t_total) + '.csv',np.array([x_node,T_t]).T, delimiter=',', header='x,T', comments='')



# print( t_total,',', T_t[0],',',x_node[0],',',T_t[10],',',x_node[10],',',T_t[18],',',x_node[18].T)
print("Ceff =", Cp_eff)
# End time integration loop

t_step check = 1.316 s Actual Timestep is = 0.25
Ceff = 7226.756756756757
